In [3]:
from konlpy.tag import Twitter
import gensim

In [4]:
def read_data(filename):    
    with open(filename, 'r',encoding='utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]        
        data = data[1:]   # header 제외 #    
    return data 


In [5]:
twitter = Twitter() 

def tokenize(doc):

    return ['/'.join(t) for t in twitter.pos(doc, norm=True, stem=True)]


In [7]:
def word2vec_train(tokens):
    model = gensim.models.Word2Vec(size=300, sg=1, alpha=0.025, min_alpha=0.025, seed=410)
    model.build_vocab(tokens)
    model.train(tokens, model.corpus_count, epochs=model.epochs)
    model.save('word2vec.model')

In [ ]:
def word2vec_load():
    model = gensim.models.word2vec.Word2Vec.load('word2vec.model')
    return model

In [8]:
def token2vec(model, doc):
    word_vec = []
    word_num = 0
    unk_num = 0
    for sent in doc:
        sub = []
        for word in sent:
            word_num += 1
            if(word in model.wv.vocab):
                sub.append(model.wv[word])
            else:
                unk_num += 1
                sub.append(np.zeros(300)) # used for unknown
        word_vec.append(sub)
    print('word_num :', word_num, 'unk_num :', unk_num)
    return word_vec

In [9]:
def zero_padding(max_length, seq, embedding_size):
    zero_pad = np.zeros((max_length) * embedding_size)
    seq_flat = np.reshape(seq, [-1])
    if(len(seq) > max_length):
        zero_pad[0:] = seq_flat[0:max_length * embedding_size]
    else:
        zero_pad[0:len(seq) * embedding_size] = seq_flat
    return zero_pad
